In [1]:
import mysql.connector
from tabulate import tabulate

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='hospital'
)

cursor = connection.cursor()
if connection.is_connected():
    print("Connection opened")

# Placeholder for get_unique_opdno function, replace it with your actual implementation
def get_unique_opdno(cursor):
    # Your logic to generate a unique OPD number
    pass

def insert_record_OPD(cursor, connection):
    try:
        opdno = int(input('Enter the OPD number: '))
        query = "SELECT * FROM opd WHERE opdno = %s"
        cursor.execute(query, (opdno,))

        result = cursor.fetchone()

        if result:
            print(f"OPD number {opdno} already exists. Please use a unique OPD number.")
            return

        pname = input("Enter patient name: ").lower()  # Applying lower() to patient name
        doa = input("Enter appointment date (YYYY-MM-DD): ")

        cursor.execute("SELECT padd, pmob FROM patient WHERE pname = %s", (pname,))
        existing_record = cursor.fetchone()
        padd = existing_record[0] if existing_record else input("Enter patient address: ")
        pmob = existing_record[1] if existing_record else input("Enter patient mobile number: ")

        cursor.execute('SELECT * FROM doctor')
        doctor_records = cursor.fetchall()

        if doctor_records:
            headers = ["DID", "DNAME", "DEPARTMENT"]
            doctor_table = tabulate(doctor_records, headers=headers, tablefmt="grid")
            print("Doctor List:")
            print(doctor_table)
        else:
            print("No records found")

        doctor_choice = int(input("Select the doctor (enter the number): "))
        dname = doctor_records[doctor_choice - 1][1]

        symptom = input("Enter symptom: ")

        cursor.execute("SELECT fee FROM opd WHERE opdno = %s", (opdno,))
        existing_fee_record = cursor.fetchone()
        fee = existing_fee_record[0] if existing_fee_record else input("Enter doctor fee: ")

        cursor.execute('INSERT INTO OPD(OPDNO, PNAME, DNAME, FEE) VALUES (%s, %s, %s, %s)', (opdno, pname, dname, fee))
        connection.commit()
        print('Record inserted into OPD successfully.')

        cursor.execute('INSERT INTO appoint(OPDNO, PNAME, DOA, DNAME, SYMPTOM) VALUES (%s, %s, %s, %s, %s)', (opdno, pname, doa, dname, symptom))
        connection.commit()
        print('Record inserted into appoint successfully.')

        cursor.execute('INSERT INTO patient(PNAME, PADD, PMOB) VALUES (%s, %s, %s)', (pname, padd, pmob))
        connection.commit()
        print('Record inserted into patient successfully.')

        cursor.execute('SELECT * FROM OPD WHERE OPDNO = %s', (opdno,))
        opd_records = cursor.fetchall()

        if opd_records:
            headers = ["OPDNO", "PNAME", "DNAME", "FEE"]
            opd_table = tabulate(opd_records, headers=headers, tablefmt="grid")
            print(opd_table)
        else:
            print("No records found in the OPD table")

    except mysql.connector.Error as e:
        print(f"MySQL Error: {e}")
    except Exception as ex:
        print(f"An unexpected error occurred: {ex}")

def update_record_OPD(cursor, connection):
    try:
        OPDNO = int(input('Enter the out-patient department no to update: '))
        NEW_PNAME = input('Enter the New Patient Name: ').lower()  # Applying lower() to new patient name
        NEW_DNAME = input('Enter the New Doctor Name: ')
        NEW_FEE = int(input('Enter the Doctor New Fee: '))

        cursor.execute('UPDATE OPD SET PNAME=%s, DNAME=%s, FEE=%s WHERE OPDNO=%s', (NEW_PNAME, NEW_DNAME, NEW_FEE, OPDNO))
        connection.commit()
        print('Record Update Successfully.')

        query = 'SELECT * FROM OPD WHERE OPDNO = %s'
        cursor.execute(query, (OPDNO,))
        updated_record = cursor.fetchall()

        if updated_record:
            headers = [desc[0] for desc in cursor.description]
            print(tabulate(updated_record, headers=headers, tablefmt="grid"))
        else:
            print("No records found.")

    except mysql.connector.Error as e:
        print(f'MySQL Error: {e}')
    except Exception as ex:
        print(f'An unexpected error occurred: {ex}')

def delete_record_OPD(cursor, connection):
    try:
        OPDNO = int(input('Enter the OPDNO Number: '))
        cursor.execute('DELETE FROM OPD WHERE OPDNO = %s', (OPDNO,))
        records = cursor.fetchall()
        if records:
            headers = [desc[0] for desc in cursor.description]
            print(tabulate(records, headers=headers, tablefmt="grid"))
    except mysql.connector.Error as e:
        print(f'MySQL Error: {e}')
    except Exception as ex:
        print(f'An unexpected error occurred: {ex}')

def search_record_OPD(cursor, connection):
    try:
        OPDNO = int(input('Enter the OPD Number to search: '))
        cursor.execute('SELECT * FROM OPD WHERE OPDNO = %s', (OPDNO,))
        records = cursor.fetchall()
        if records:
            headers = [desc[0] for desc in cursor.description]
            print(tabulate(records, headers=headers, tablefmt="grid"))
        else:
            print(f"No records found for OPD Number {OPDNO}.")

    except mysql.connector.Error as e:
        print(f'MySQL Error: {e}')
    except Exception as ex:
        print(f'An unexpected error occurred: {ex}')


Connection opened
